# C# Generics / Обобщенное программирование

Мы уже знаем о перегрузках функций:


```c#
public static int Add(int a, int b)
{
    return a + b;
}

public static double Add(double a, double b)
{
    return a + b;
}

public static decimal Add(decimal a, decimal b)
{
    return a + b;
}


И потом вот так используем. Пользователю класса все равно конечно, но для каждого типа писать один и тот же код...
```c#
    int aInt = 2, bInt = 1;
    var resultInt = Add(aInt, bInt);

    double aDouble = 2, bDouble = 1;
    var resultDouble = Add(bDouble, aDouble);

Как можно заметить, функция простая, делается все одинаково, хочется все засунуть под шаблон, как это было в плюсах.


```c#
public static T Add<T>(T x, T y)
{
    return x + y; // Error CS0019 Operator '+' cannot be applied to operands of type 'T' and 'T'
}

Попробуем починить это.
```c#
public static T Add<T>(T x, T y) where T: INumber<T>
{
    return x + y;
}

Помимо ограничений на реализуемый интерфейс есть еще следующие:

```c#
public T Foo(T x) where T : struct
public T Foo(T x) where T : class
public T Foo(T x) where T : new()
public T Foo(T x) where T : BaseClass
public T Foo(T x, U y) where T : U

Также есть эти же версии для nullable типов.
Еще их можно комбинировать друг с другом:

```c#
public static void Foo<T>(T x) where T: IComparable<T>, ISomeOtherInteface<T>
{
    //Do something using IComparable<T> and ICloneable methods.
}

## Type Inference

```c#
public class Base { }
public class Derived : Base { }
public class AnotherDerived : Base { }

...

public static void Foo<T>(T a, T b)
{
    //Do something
}

var baseClass = new Base();
var derivedClass = new Derived();
var secondDerived = new AnotherDerived();

Foo(derivedClass, derivedClass);        // Вызывает Foo<Derived>
Foo(baseClass, derivedClass);           // Вызывает Foo<Base>
Foo(secondDerived, derivedClass);       // Не может самостоятельно вывести тип
Foo<Base>(secondDerived, derivedClass); // Мы самостоятельно в таком случае выводим тип.	

```

***Спросить почему не работает так***

## Как работают дженерики
Тут про jit компиляцию
https://sharplab.io/#gist:85b176c9a208f32fd9d85b65391afd87

## Подводные камни

```c#
public static void FooBroken<T>() // will not compile
{
    //Do something
    T val = null;
    //Do something else
    return;
}

public static void FooFixed<T>() (2) where T: class
{
    //Do something
    T val = (1)default(T);
    //Do something else
    return;
}

public static void FooStruct<T>() where T: struct , new() // The 'new()' constraint cannot be used with the 'struct' constraint
{
    //Do Something
}

public static void FooDerived<T>() where T: BaseClassA, BaseClassB
{
    //Do Something
}

## Инвариантность / Ковариантность / Контрвариантность 

1) ИНВАРИАНТНОСТЬ :: Параметр-тип не может изменяться. Любой дженерик является им по умолчанию
```c#
public class BaseClass { }
public class DerivedClass : BaseClass { }

public interface MyListInterface<T> { }
public class MyList<T> : MyListInterface<T> { }

public static void Example()
{
    MyListInterface<BaseClass> integers = new MyList<BaseClass>(); // OK
    MyListInterface<BaseClass> integers = new MyList<DerivedClass>(); // Compilation Error
}
```
2) КОВАРИАНТНОСТЬ :: Параметр-тип может быть неявно преобразован от дочернего класса к базовому. Ключевое слово out. \
Обобщенный параметр с данным модификатором может только быть передан в качестве возврата функции.
```c#
public interface IEnumerable<out T> : IEnumerable

public static void Example()
{
    List<DerivedClass> derivedClasses = new List<DerivedClass>();

    ExampleFuncEnumerable(derivedClasses); // OK IEnumerable is covariant
    ExampleFuncList(derivedClasses); // Compilation Error, List is invariant
}

public static void ExampleFuncEnumerable(IEnumerable<BaseClass> baseClasses)
{
    //DO SOMETHING
}

public static void ExampleFuncList(List<BaseClass> baseClasses)
{
    //DO SOMETHING
}
```
3) КОНТРВАРИАНТНОСТЬ :: Параметр тип может быть неявно преобразован от базового класса к унаследованному. Ключевое слово in \
Обобщенный параметр с данным модификатором может только быть передан в качестве аргумента.
```c#
abstract class Shape
{
    public virtual double Area { get { return 0; }}
}

class Circle : Shape
{
    private double r;
    public Circle(double radius) { r = radius; }
    public double Radius { get { return r; }}
    public override double Area { get { return Math.PI * r * r; }}
}

class ShapeAreaComparer : System.Collections.Generic.IComparer<Shape>
{
    int IComparer<Shape>.Compare(Shape a, Shape b)
    {
        if (a == null) return b == null ? 0 : -1;
        return b == null ? 1 : a.Area.CompareTo(b.Area);
    }
}

class Program
{
    static void Main()
    {
        //Хоть мы и определили наш компаратор для шейпа, тем
        //не менее мы можем его использовать и в качестве
        //IComparer<Circle>, так как имеем на это полное право,
        //ведь работаем только с базовым классом.
        SortedSet<Circle> circlesByArea =
            new SortedSet<Circle>(new ShapeAreaComparer()) // IComparer<Circle>
                { new Circle(7.2), new Circle(100), null, new Circle(.01) };

        foreach (Circle c in circlesByArea)
        {
            Console.WriteLine(c == null ? "null" : "Circle with area " + c.Area);
        }
    }
}
```

## Пишем свой Optional\Nullable\Lazy ... TODO TODO TODO

Добавим сначала объявление класса // Нужно расписать по частям что тут происходи
```c#
public abstract class Option<T>
{
    public static implicit operator Option<T>(T value) =>
        new Some<T>(value);

    public static implicit operator Option<T>(None none) =>
        new None<T>();

    public Option<TNew> OfType<TNew>() where TNew : class =>
    this is Some<T> some && typeof(TNew).IsAssignableFrom(typeof(T))
        ? (Option<TNew>) new Some<TNew>(some.Content as TNew)
        : None.Value;
}

public sealed class Some<T> : Option<T>, IEquatable<Some<T>>
{
    public T Content { get; }

    public Some(T value)
    {
        Content = value;
    }

    public bool Equals(Some<T>? other)
    {
        if (ReferenceEquals(null, other)) return false;
        if (ReferenceEquals(this, other)) return true;
        return EqualityComparer<T>.Default.Equals(Content, other.Content);
    }

    public override bool Equals(object? obj)
    {
        if (ReferenceEquals(null, obj)) return false;
        if (ReferenceEquals(this, obj)) return true;
        return obj is Some<T> some && Equals(some);
    }

    public override int GetHashCode()
    {
        return Content is null ? 0 : EqualityComparer<T>.Default.GetHashCode(Content);
    }

    public static bool operator ==(Some<T> a, Some<T> b) =>
        (a is null && b is null) ||
        (!(a is null) && a.Equals(b));

    public static bool operator !=(Some<T> a, Some<T> b) => !(a == b);
}

public sealed class None<T> : Option<T>,
    IEquatable<None<T>>, IEquatable<None>
{
    public override bool Equals(object? obj) =>
    !(obj is null) && ((obj is None<T>) || (obj is None));

    public override int GetHashCode() => 0;

    public bool Equals(None<T>? other) => true;

    public bool Equals(None? other) => true;

    public static bool operator ==(None<T> a, None<T> b) =>
        (a is null && b is null) ||
        (!(a is null) && a.Equals(b));

    public static bool operator !=(None<T> a, None<T> b) => !(a == b);
}

public sealed class None : IEquatable<None>
{
    public static None Value { get; } = new None();

    private None() { }

    public override bool Equals(object? obj)
    {
        return !(obj is null) && ((obj is None) || IsGenericNone(obj.GetType()));
    }

    private static bool IsGenericNone(Type type)
    {
        return type.GenericTypeArguments.Length == 1 &&
                typeof(None<>)
                .MakeGenericType(type.GenericTypeArguments[0]) == type;
    }

    public bool Equals(None? other) => true;

    public override int GetHashCode() => 0;
}
```

## Какие ограничения накладывают дженерики и другие проблемы

### 1) Нельзя использовать in/out обобщенных типов для классов и их не интерфейсных методах.
Если хотите этим пользоваться, то определяйте интерфейс.
```c#
//Error	CS1960	Invalid variance modifier. Only interface and delegate type parameters can be specified as variant.
public class ProblemClass<in F> { } // Так нельзя

public T Foo<out T, in G>(G value) where T : new() // Так тоже нельзя.
{
    return new T();
}
``` 

### Почему нам не дают делать классы ковариантными\контрвариантными.


Представим некий метафизический `List<out T>` (его не существует)

```c#
List<string> strings = new List<string>();
strings = strings.Append("hello");
strings = strings.Append("goodbye");
List<object> objects = strings; // Мы можем так сделать, ведь лист ковариантен
objects.Append(123);            // Мы все сломали )))
```

А теперь представим еще один некий метафизический `List<in T>` (его тоже не существует)

```c#
List<Base> bases = new List<Base>();
bases = bases.Push(new Base(1));
bases = bases.Push(new Base(2));
List<Derived> derived = strings;        // Мы можем так сделать, ведь лист контрвариантен
var someDerived = derived[0];           // Мы даже не подозреваем ничего
someDerived.CallFunctionOnlyInDerived() // Ой-ой
```

Сделано это было в общем-то ради типобезопасности (T[] передает привет).


### Почему в интерфейсах то можно?
На самом деле из-за того, что в интерфейсах не может быть полей (свойства != поле)

```c#
public interface IShapeRepository<in T>
{
    public T ValueProperty { set; get; } // Compilation Error, T is contrvariant. Мы не можем его геттить
}

public interface IShapeRepository<in T>
{
    public T ValueProperty { set; }
}

public class ShapeRepository<TShape> : IShapeRepository<TShape>
    where TShape: Shape
{
    public TShape ValueField;

    public TShape ValueProperty { set { ValueField = value; } }
}

...

var shapeRepo = new ShapeRepository<Shape>();
shapeRepo.ValueField = new Rect();
IShapeRepository<Circle> circleRepo = shapeRepo;  // Здесь срабатывает контрвариантность
var fakeCircle = circleRepo.ValueProperty;        // Error, ValueProperty имеет только сеттер
                                                  // Если бы у нас были "поля" в интерфейсах
                                                  // То мы бы прямоугольник вместо ожидаемого круга.
```

Если бы могли быть магическим образом создавать поля в интерфейсах, то у нас бы не было типобезобасности\
Тоже самое можете сами проверить и с ковариантностью

### 2) Нет нормальной специализации шаблонов как в тех же плюсах.
```c#
public interface ITypePrinter<T>
{
    public void PrintType();
}

public class TypePrinter<T> : ITypePrinter<T>
{
    public void PrintType()
    {
        Console.WriteLine($"Type is {typeof(T)}");
    }
}

public class StringTypePrinter : ITypePrinter<string>
{
    public void PrintType()
    {
        Console.WriteLine("My Type is string, not infered from abstract");
    }
}

///

var printer = new TypePrinter<int>();
var strPrinter = new StringTypePrinter();
var fakePrinter = new TypePrinter<string>();

printer.PrintType(); // Type is System.Int32
strPrinter.PrintType(); // My Type is string, not infered from abstract
fakePrinter.PrintType(); // Type is System.String

```

Также еще заметим некоторую интересную возможность:

```c#
private static void Display(String s)
{
    Console.WriteLine("From Adhoc");
}

private static void Display<T>(T o)
{
    Console.WriteLine("From parametrized");
}

Display("Jeff");
Display(123);
Display<(String)>("Aidan");

```
**ВОПРОС** Что выведется на экране

<details>
<summary>Ответ</summary>

* Display("Jeff"); // From Adhoc
* Display(123); // From parametrized
* Display<(String)>("Aidan"); // From parametrized

Как можно заметить, приоритет вызова параметрической функции довольно мал.

</details>

Можно тогда обойти некоторым образом эту проблему через extension методы

```c#
class TypePrinter<T>
{
    public void PrintType()
    {
        Console.WriteLine($"Type is {typeof(T)}");
    }
}

static class MyTypePrinterSpecialization
{
    public static void PrintTypeSpecialization(this TypePrinter<string> printer)
    {
        Console.WriteLine("My Type is string, not infered from abstract");
    }

    public static void PrintTypeSpecialization<T>(this TypePrinter<T> printer)
    {
        Console.WriteLine($"Type is {typeof(T)}");
    }
}

...

var printer = new TypePrinter<int>();
var fakePrinter = new TypePrinter<string>();

printer.PrintTypeSpecialization(); // Type is System.Int32
fakePrinter.PrintTypeSpecialization(); // My Type is string, not infered from abstract
```

Получили что хотели. Однако это странно хранить методы в экстеншенах.
Получается все по дэбильному, но если надо то можно этим пользоваться.

Но будет это ломаться в таком случае
```c#
Print(new TypePrinter<int>());
Print(new TypePrinter<string>());
Print(new StringTypePrinter());

...

public static void Print<T>(TypePrinter<T> printer)
{
    printer.PrintTypeSpecialization();
}

//Выведет в терминал
//Type is System.Int32
//Type is System.String
//Type is System.String
```

### 3) Нельзя в качестве аргумента дженерика использовать литералы 
Или другими словами дженерик не может быть открытым типом изначально \
то есть иметь вместо обобщенного типа явно существующий объект.

```c#
class Matrix<int W, int H> { ... } // Compilation Error
```

## Новые Фичи

### static abstract/virtual + Generic Math

Раньше до 7 дотнета нельзя было так писать:

```c#
public interface IUnit : IUnitOperandLeft, IUnitOperandRight
{
    public static IUnit operator +(IUnit left, IUnit right)
        => left.Add(right);

    public static IUnit operator -(IUnit left, IUnit right)
        => left.Subtract(right);

    public static IUnit operator *(IUnit left, IUnit right)
        => left.Multiply(right);

    public static IUnit operator /(IUnit left, IUnit right)
        => left.Divide(right);
}
```

А теперь можно. Также еще можно имплементировать их в самом классе, пометив их словом abstract

```c#
public interface IUnit : IUnitOperandLeft, IUnitOperandRight
{
    public abstract static IUnit operator +(IUnit left, IUnit right)

    public abstract static IUnit operator -(IUnit left, IUnit right)

    public abstract static IUnit operator *(IUnit left, IUnit right)

    public abstract static IUnit operator /(IUnit left, IUnit right)
}
```

И при этом это все может работать с дженериками

```c#
public class Point(double X, double Y) 
    : IAdditionOperators<Point, Point, Point> // Реализуем специальный для этого интерфейс
{
    public double X { get; } = X;
    public double Y { get; } = Y;

    public static Point operator +(Point left, Point right)
    {
        return new Point(left.X + right.X, left.Y + right.Y);
    }

    public void TestMath()
    {
        var points = new List<Point> { new Point(0, 0), new Point(1, 1), new Point(2, 2), };

        var bigPoint = points.SumAll();
    }
}

...

// Почему-то они так и не сделали нормальных для этого методов linq
// Придется нам это сделать самим.
public static class SumEnumerableExtension
{
    public static T SumAll<T>(this IEnumerable<T> values)
        where T: IAdditionOperators<T, T, T>
    {
        var value = default(T);

        foreach (var item in values)
        {
            value += item;
        }

        return value;
    }
}
```

## Секция с вопросами TODO ###